### DK Salaries Optimizer

In [104]:
import pandas as pd
import numpy as np
from pulp import *

In [105]:
def get_pos(name):
    l = name.split('\xa0')
    return l[-1]

'Giannis Antetokounmpo PF'.split('\xa0')

['Giannis Antetokounmpo', 'PF']

In [106]:
dk_data = pd.read_csv("numberfire_nfl.csv")
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To
0,Christian McCaffrey RB,23.8,"$10,000",2.38,0.00/0.00,0.00,0.00,0.00,17.08,76.75,0.50
1,LV @ CARSun 1:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lamar Jackson QB,23.5,"$8,100",2.90,20.17/31.31,221.70,1.81,0.65,10.69,54.11,0.47
3,CLE @ BAL GTDSun 1:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Matt Ryan QB,22.5,"$6,700",3.36,26.05/39.49,307.49,1.72,0.72,2.08,8.24,0.06


In [107]:
# drop rows that have nan
dk_data = dk_data.dropna(axis=0, how='any')
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To
0,Christian McCaffrey RB,23.8,"$10,000",2.38,0.00/0.00,0.00,0.00,0.00,17.08,76.75,0.50
2,Lamar Jackson QB,23.5,"$8,100",2.90,20.17/31.31,221.70,1.81,0.65,10.69,54.11,0.47
4,Matt Ryan QB,22.5,"$6,700",3.36,26.05/39.49,307.49,1.72,0.72,2.08,8.24,0.06
6,Russell Wilson QB,20.7,"$7,000",2.95,21.45/32.55,256.12,1.96,0.54,4.49,23.55,0.14
8,Michael Thomas WR,20.1,"$9,000",2.23,0.00/0.00,0.00,0.00,0.00,0.06,0.38,0.00


In [108]:
# Get the fields that are useful
#dk_data = dk_data[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame']]
dk_data['Roster Position'] = dk_data.apply(lambda x: get_pos(x['Player']),axis=1)

# change $s into int
dk_data['Cost'] = dk_data['Cost'].replace('[\$,]', '', regex=True).astype(float)

dk_data.head(10)

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To,Roster Position
0,Christian McCaffrey RB,23.8,10000.0,2.38,0.00/0.00,0.00,0.00,0.00,17.08,76.75,0.50,RB
2,Lamar Jackson QB,23.5,8100.0,2.90,20.17/31.31,221.70,1.81,0.65,10.69,54.11,0.47,QB
4,Matt Ryan QB,22.5,6700.0,3.36,26.05/39.49,307.49,1.72,0.72,2.08,8.24,0.06,QB
6,Russell Wilson QB,20.7,7000.0,2.95,21.45/32.55,256.12,1.96,0.54,4.49,23.55,0.14,QB
8,Michael Thomas WR,20.1,9000.0,2.23,0.00/0.00,0.00,0.00,0.00,0.06,0.38,0.00,WR
10,Josh Allen QB,19.3,6500.0,2.97,18.78/31.43,208.19,1.30,0.75,8.99,39.76,0.48,QB
12,Dalvin Cook RB,19.2,7900.0,2.42,0.00/0.00,0.00,0.00,0.00,16.94,77.30,0.60,RB
14,Cam Newton QB,19.0,6100.0,3.11,20.09/32.47,233.46,1.59,0.87,5.47,24.18,0.32,QB
16,Kyler Murray QB,18.9,6400.0,2.95,22.88/35.49,254.25,1.53,0.71,4.64,23.32,0.17,QB
18,Alvin Kamara RB,18.8,7200.0,2.62,0.00/0.00,0.00,0.00,0.00,13.80,61.38,0.58,RB


In [109]:
injured_players = ['Kenny Golladay\xa0WR', 'DeAndre Hopkins\xa0WR', 'test']

In [110]:
dk_data = dk_data[~dk_data['Player'].isin(injured_players)]

In [111]:
# Roster Positions for NBA 
nfl_roster_positions = ['QB', 'RB', 'WR', 'TE', 'DST']

SALARY_CAP = 50000
MAX_PLAYERS = 9


def make_vars(name, position, roster_position):
    if position in roster_position:
        return LpVariable(name + '_' + position,lowBound=0, cat='Binary')
    return 0

for position in nfl_roster_positions:
    dk_data[position] = dk_data.apply(lambda x: make_vars(x['Player'],position, x['Roster Position']),axis=1)

    #[1 if position in x else 0 for x in dk_data['Roster Position']]

In [112]:
prob = LpProblem("OptimizeDK",LpMaximize)

In [113]:
names = dk_data['Player']
salaries = dk_data['Cost']
projected_points = dk_data['FP']

In [114]:
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To,Roster Position,QB,RB,WR,TE,DST
0,Christian McCaffrey RB,23.8,10000.0,2.38,0.00/0.00,0.00,0.00,0.00,17.08,76.75,0.50,RB,0,Christian_McCaffrey RB_RB,0,0,0
2,Lamar Jackson QB,23.5,8100.0,2.90,20.17/31.31,221.70,1.81,0.65,10.69,54.11,0.47,QB,Lamar_Jackson QB_QB,0,0,0,0
4,Matt Ryan QB,22.5,6700.0,3.36,26.05/39.49,307.49,1.72,0.72,2.08,8.24,0.06,QB,Matt_Ryan QB_QB,0,0,0,0
6,Russell Wilson QB,20.7,7000.0,2.95,21.45/32.55,256.12,1.96,0.54,4.49,23.55,0.14,QB,Russell_Wilson QB_QB,0,0,0,0
8,Michael Thomas WR,20.1,9000.0,2.23,0.00/0.00,0.00,0.00,0.00,0.06,0.38,0.00,WR,0,0,Michael_Thomas WR_WR,0,0


In [115]:
# The objective function is added to 'prob' first
# OBJ FUNCTION
prob += lpSum([dk_data[i] * projected_points for i in nfl_roster_positions]), 'Projected Points'

In [116]:
# CONSTRAINTS

# Stay under salary cap
prob += lpSum(salaries * dk_data[i] for i in nfl_roster_positions) <= SALARY_CAP

# Stay under Num Players
prob += lpSum(dk_data[i] for i in nfl_roster_positions) <= MAX_PLAYERS

# Select one of each position (flexibility for flex except QB and DST)
prob += lpSum([dk_data['QB']]) == 1
prob += lpSum([dk_data['RB']]) >= 2
prob += lpSum([dk_data['WR']]) >= 3
prob += lpSum([dk_data['TE']]) >= 1
prob += lpSum([dk_data['DST']]) == 1

# Don't use same player twice
#for index, row in dk_data.iterrows():
#    prob += lpSum(row[i] for i in nfl_roster_positions) <= 1


In [117]:
prob

OptimizeDK:
MAXIMIZE
4.2*A.J._Dillon RB_RB + 12.4*A.J._Green WR_WR + 14.9*Aaron_Jones RB_RB + 16.9*Aaron_Rodgers QB_QB + 2.8*Adam_Shaheen TE_TE + 14.5*Adam_Thielen WR_WR + 1.9*Adam_Trautman TE_TE + 5.1*Adrian_Peterson RB_RB + 0.7*Alec_Ingold RB_RB + 0.4*Alex_Armah RB_RB + 1.2*Alex_Erickson WR_WR + 0.7*Alexander_Hollins WR_WR + 6.6*Alexander_Mattison RB_RB + 9.3*Allen_Lazard WR_WR + 15.3*Allen_Robinson WR_WR + 18.8*Alvin_Kamara RB_RB + 0.6*Ameer_Abdullah RB_RB + 1.0*Andre_Roberts WR_WR + 5.7*Andy_Isabella WR_WR + 10.1*Anthony_Miller WR_WR + 7.0*Antonio_Gandy_Golden WR_WR + 9.9*Antonio_Gibson RB_RB + 5.7*Arizona_DST_DST + 0.5*Ashton_Dulin WR_WR + 6.0*Atlanta_DST_DST + 4.2*Auden_Tate WR_WR + 0.6*Austin_Carr WR_WR + 17.1*Austin_Ekeler RB_RB + 8.4*Austin_Hooper TE_TE + 15.6*Baker_Mayfield QB_QB + 7.8*Baltimore_DST_DST + 0.4*Bennie_Fowler WR_WR + 3.5*Bisi_Johnson WR_WR + 8.4*Boston_Scott RB_RB + 10.9*Brandon_Aiyuk WR_WR + 1.1*Brandon_Powell WR_WR + 1.4*Braxton_Berrios WR_WR + 9.0*Breshad_Per

In [118]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [119]:
prob.status

1

In [120]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [121]:
print("Total Estimated Points = ", value(prob.objective))


Total Estimated Points =  132.9


In [122]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue == 1:
        print(v.name)
        v.getName

Alvin_Kamara RB_RB
Calvin_Ridley WR_WR
Chris_Herndon TE_TE
Davante_Adams WR_WR
DeSean_Jackson WR_WR
Josh_Jacobs RB_RB
Matt_Ryan QB_QB
New_York_Jets_DST_DST
Terry_McLaurin WR_WR
